In [26]:
from collections import defaultdict
from sklearn import linear_model
import csv
import random
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import precision_score,recall_score


## Data Process

In [2]:
def readCSV(path):
    f = open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d

In [3]:
def get_feature(path):
    recipe_dict = {}
    for d in tqdm(readCSV(path)):
        recipe_dict[d['id']]= d
#         ,d['name_tokens'],d['ingredient_tokens'],d['steps_tokens'],d['techniques'],d['calorie_level'],d['ingredient_ids']
#         recipe_count_dict[recipe] += 1
#         user_recipe_dict[user].add(recipe)
#         train_cooked += 1
    return recipe_dict

In [4]:
recipe_dict = get_feature("data/PP_recipes.csv")

178265it [00:03, 45905.47it/s]


In [5]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

r_recipes = pd.read_csv('data/RAW_recipes.csv')
p_recipes = pd.read_csv('data/PP_recipes.csv')
# test = pd.read_csv('data/interactions_test.csv')
# train = pd.read_csv('data/interactions_train.csv')
# validation = pd.read_csv('data/interactions_validation.csv')

In [6]:
r_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [7]:
p_recipes

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"
...,...,...,...,...,...,...,...,...
178260,323143,76862,"[40480, 6444, 1964, 9369, 486, 569, 17551, 40481]","[[8780], [11835, 1762, 4465, 31494], [6812], [...","[40480, 40482, 729, 2525, 715, 485, 26641, 404...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[840, 208, 2499, 2683, 1925, 335, 1511]"
178261,149114,145962,"[40480, 17027, 24715, 974, 11877, 40481]","[[6812], [5940], [30645, 4785, 6821], [6953], ...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[2499, 4717, 1168, 6270, 6324, 7040]"
178262,34200,65066,"[40480, 12187, 11434, 1738, 2627, 40481]","[[6167, 20930, 510], [1353], [15022, 6953], [6...","[40480, 40482, 500, 246, 1719, 5024, 240, 2366...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",2,"[2378, 7655, 3219, 2320, 5168, 5319, 4189, 268..."
178263,30618,77358,"[40480, 870, 488, 1325, 519, 2220, 2417, 488, ...","[[12395, 38308, 40118], [3137, 15022], [30878,...","[40480, 40482, 562, 481, 10734, 240, 23667, 58...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[5627, 2807, 5412, 3399, 7979, 1093, 1257, 780..."


In [8]:
calorie_level = list(p_recipes.calorie_level)

In [9]:
train_size = int(0.8 * len(p_recipes))
p_recipes_train = p_recipes[:train_size]
calorie_level_train = p_recipes.calorie_level[:train_size]
p_recipes_test = p_recipes[train_size:]
calorie_level_test = p_recipes.calorie_level[train_size:]

In [10]:
p_recipes_train

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"
...,...,...,...,...,...,...,...,...
142607,289133,62432,"[40480, 1527, 478, 36, 6060, 21961, 6047, 488,...","[[21961, 7111], [15022, 6953], [21453], [8780]...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[2516, 3219, 5010, 840, 1723, 7801, 7213, 2683..."
142608,135201,6959,"[40480, 13731, 21278, 40481]","[[21190], [6444], [1353], [9230, 789, 607], [1...","[40480, 40482, 2698, 488, 21662, 11125, 40478,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6846, 6906, 7655, 1833, 7449, 4284]"
142609,264171,68193,"[40480, 38450, 5082, 1301, 16201, 22942, 40481]","[[6167, 1301, 11007], [15022, 796, 964], [1941...","[40480, 40482, 21662, 488, 2042, 481, 1301, 11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",1,"[2218, 3203, 5010, 1986, 1329, 3502, 2320, 393..."
142610,282958,37415,"[40480, 265, 263, 1257, 539, 252, 5390, 1866, ...","[[4017], [33192, 1101, 25388, 39727], [11182],...","[40480, 40482, 2525, 481, 18775, 4017, 668, 48...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[3723, 6384, 710, 5058, 5038, 6846, 6731, 3824..."


## techniques

In [15]:
technique_feature_train = list(eval(technique) for technique in p_recipes_train.techniques)
technique_feature_test = list(eval(technique) for technique in p_recipes_test.techniques)

In [16]:
len(calorie_level)

178265

In [18]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=800).fit(technique_feature_train, calorie_level_train)
LR.predict(technique_feature_test)
round(LR.score(technique_feature_test, calorie_level_test), 4)


0.4474

In [ ]:
SVM = svm.SVC(decision_function_shape="ovo").fit(technique_feature_train, calorie_level_train)
SVM.predict(technique_feature_test)
round(SVM.score(technique_feature_test, calorie_level_test), 4)

In [18]:
RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(technique_feature_train, calorie_level_train)
RF.predict(technique_feature_test)
round(RF.score(technique_feature_test, calorie_level_test), 4)


0.4528

In [22]:
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(150, 10), random_state=1, max_iter=800).fit(technique_feature_train, calorie_level_train)
NN.predict(technique_feature_test)
round(NN.score(technique_feature_test, calorie_level_test), 4)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.4471

In [ ]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(technique_feature_train, calorie_level_train)
predictions = clf.predict_proba(technique_feature_test)

/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## ingredient_ids

In [11]:
ingredient_id_train = [eval(ingredient_ids) for ingredient_ids in p_recipes_train.ingredient_ids]
ingredient_id_test = [eval(ingredient_ids) for ingredient_ids in p_recipes_test.ingredient_ids]

In [12]:
ingredient_count = defaultdict(int)
for ingredient_ids in p_recipes_train.ingredient_ids:
    for ingredient in eval(ingredient_ids):
        ingredient_count[ingredient] += 1
ingredients_sorted = sorted(list(ingredient_count.items()), key= lambda x:x[1], reverse=True)[:500]
    
def feat(ingredient_id):
    feature = []
    for ingredient, count in ingredients_sorted:
        if ingredient in ingredient_id:
            feature.append(1)
        else:
            feature.append(0)
    return feature

In [17]:
onehot_ingredient_id_train =  np.array([feat(ingredient_id) for ingredient_id in ingredient_id_train])

In [18]:
onehot_ingredient_id_test =  np.array([feat(ingredient_id) for ingredient_id in ingredient_id_test])

In [85]:
len(onehot_ingredient_id_train[1])

500

In [28]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=800).fit(onehot_ingredient_id_train, calorie_level_train)
LR_predictions = LR.predict(onehot_ingredient_id_test)
# round(LR.score(onehot_ingredient_id_test, calorie_level_test), 4)


0.5189465122149609


In [37]:
print(precision_score(calorie_level_test, LR_predictions,average='macro'))

0.5098733810358201


In [30]:
RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(onehot_ingredient_id_train, calorie_level_train)
RF_predictions = RF.predict(onehot_ingredient_id_test)
round(RF.score(onehot_ingredient_id_test, calorie_level_test), 4)


0.46108321880346675


In [35]:
print(precision_score(calorie_level_test, RF_predictions,average='macro'))

0.5595758742462492


In [31]:
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1, max_iter=1000).fit(onehot_ingredient_id_train, calorie_level_train)
NN_predictions = NN.predict(onehot_ingredient_id_test)
round(NN.score(onehot_ingredient_id_test, calorie_level_test), 4)


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.4602417748857039


In [33]:
print(precision_score(calorie_level_test, NN_predictions,average='macro'))


0.4480248426248102


In [15]:
clf = xgb.XGBClassifier()


In [19]:
clf.fit(onehot_ingredient_id_train, calorie_level_train)


XGBClassifier(objective='multi:softprob')

In [22]:
predictions = clf.predict(onehot_ingredient_id_test)

In [32]:
print(precision_score(calorie_level_test, predictions,average='macro'))

0.509772892303256


In [25]:
predictions[100:200]

array([1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 2, 2, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0])

## Ingredient: 1000

In [41]:
ingredient_id_train = [eval(ingredient_ids) for ingredient_ids in p_recipes_train.ingredient_ids]
ingredient_id_test = [eval(ingredient_ids) for ingredient_ids in p_recipes_test.ingredient_ids]

In [42]:
ingredient_count = defaultdict(int)
for ingredient_ids in p_recipes_train.ingredient_ids:
    for ingredient in eval(ingredient_ids):
        ingredient_count[ingredient] += 1
ingredients_sorted = sorted(list(ingredient_count.items()), key= lambda x:x[1], reverse=True)[:1000]
    
def feat(ingredient_id):
    feature = []
    for ingredient, count in ingredients_sorted:
        if ingredient in ingredient_id:
            feature.append(1)
        else:
            feature.append(0)
    return feature
onehot_ingredient_id_train =  np.array([feat(ingredient_id) for ingredient_id in ingredient_id_train])
onehot_ingredient_id_test =  np.array([feat(ingredient_id) for ingredient_id in ingredient_id_test])

In [43]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=800).fit(onehot_ingredient_id_train, calorie_level_train)
LR_predictions = LR.predict(onehot_ingredient_id_test)
print(precision_score(calorie_level_test, RF_predictions,average='macro'))

RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0, max_iter=1000).fit(onehot_ingredient_id_train, calorie_level_train)
RF_predictions = RF.predict(onehot_ingredient_id_test)
print(precision_score(calorie_level_test, RF_predictions,average='macro'))

clf = xgb.XGBClassifier()
clf.fit(onehot_ingredient_id_train, calorie_level_train)
predictions = clf.predict(onehot_ingredient_id_test)
print(precision_score(calorie_level_test, predictions,average='macro'))

0.5595758742462492
0.5816747757209456


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.4529989612256479
0.5246378647916056


In [45]:

NN = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1, max_iter=2000).fit(onehot_ingredient_id_train, calorie_level_train)
NN_predictions = NN.predict(onehot_ingredient_id_test)
print(precision_score(calorie_level_test, NN_predictions,average='macro'))


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.4495948752612106


## GradientBoostingClassifier¶

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [52]:
class GBDT():
    def __init__(self, args = {}, random_state=42, verbose=0):
        self.random_state = random_state
        self.args = args
        self.verbose = verbose
        self.models = []
        self.func = GradientBoostingClassifier
        
        self.feature_importances = {}
    def fit(self, X_train_, y_train_, cv=1, analyze=False):
        self.models = []
        classification_train(self,X_train_, y_train_, cv,func=self.func,analyze=analyze)
            
    def predict(self, X_test, verbose=0):
        preds = np.zeros(len(X_test))
        if verbose!=0: print("[",end="")
        for model in self.models:
            if verbose!=0:print("-",end="")
            preds += model.predict(X_test)
        if verbose!=0:print("]")
        preds /= len(self.models)
        return [int(round(i)) for i in preds]
    
    def evaluate(self, X_test, y_test, return_score=True):
        preds = self.predict(X_test)
        acc = accuracy_score(y_test, preds)
        print("Accuracy: {:5f}".format(acc))
        if return_score:
            return acc
        
    def copy(self):
        return self.func(random_state=self.random_state, verbose=self.verbose, **self.args)

def classification_train(this, X_train_, y_train_, cv=1, func=None, analyze=False):
    v = 1 if this.verbose == 2 else 0
    if cv == 1:
        this.models.append(func(random_state=this.random_state, verbose=v, **this.args))
        this.models[0].fit(X_train_,y_train_)
        preds_train = [round(i) for i in this.models[0].predict(X_train_)]
        print("Train Accuracy: {:5f}".format(accuracy_score(y_train_, preds_train)))
        if analyze:
            this.feature_importances = dict(zip(this.models[0].feature_importances_, X_train_.columns.values))
            this.feature_importances = sorted(this.feature_importances.items(), key=operator.itemgetter(0),reverse=True)[:20]
            plt.figure(figsize=(30,10))
            plt.bar([x[1] for x in this.feature_importances],[x[0] for x in this.feature_importances],color="red")
            plt.title("Feature Importances")
            plt.show()
    else:
        importances = np.zeros(len(X_train.columns))
        val_accuracies = []
        train_accuracies = []
        size = len(X_train)
        train_size = 1 - 1 / cv
        
        for i in range(cv):
            this.models.append(func(random_state=this.random_state, verbose=v, **this.args))
            if this.verbose != 0: print("{}/{}".format(i+1,cv))
            X_fold, X_val, y_fold, y_val = train_test_split(X_train_, y_train_, train_size=train_size)
            if this.verbose != 0: print("Training on {} instances, Validating on {} instances".format(len(X_fold),len(X_val)))
            this.models[i].fit(X_fold,y_fold)
            val_preds = [round(i) for i in this.models[i].predict(X_val)]
            val_accuracies.append(accuracy_score(y_val,val_preds))
            importances += this.models[i].feature_importances_
            if this.verbose != 0: print("Validation Accuracy: {:5f}\n".format(accuracy_score(y_val,val_preds)))
            
        if analyze:
            this.feature_importances = dict(zip(importances, X_train.columns.values))
            this.feature_importances = sorted(this.feature_importances.items(), key=operator.itemgetter(0),reverse=True)
            print("Mean Validation Accuracy: {:5f}".format(np.mean(val_accuracies)))
            f,axs = plt.subplots(1, 2,figsize=(30,10))
            axs = axs.flatten()
            axs[0].bar([x[1] for x in this.feature_importances],[x[0] for x in this.feature_importances],color="red")
            axs[0].set_title("Feature Importances")
            plt.setp(axs[0].xaxis.get_majorticklabels(), rotation=90)
            
            axs[1].hist(val_preds)
            axs[1].set_title("Prediction Distribution")
            plt.show()

class GridSearch():
    def __init__(self, model, params, verbose=0):
        self.verbose = verbose
        self.model = model
        self.best_params = None
        self.params = params
        self.features = list(params.keys())
        self.combinations = list(product(*list(params.values())))
        
    def fit(self, X, y, train_size=0.8, cv=1, stratify=True, custom=True):
        print("Total Combinations: " + str(len(self.combinations)))
        if cv == 1:
            accuracies = []
            highest, count = 0, 0
            if stratify: X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=train_size,stratify=y,shuffle=True)
            else: X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=train_size,shuffle=True)
            start = datetime.datetime.now()
            for combination in self.combinations:
                count += 1
                model = self.model(args=dict(zip(self.features,combination)))
                model.fit(X_train, y_train)
                preds = model.predict(X_test)
                acc = accuracy_score(y_test,preds)
                accuracies.append(acc)
                if acc > highest:
                    self.best_params = dict(zip(self.features,combination))
                    highest = acc
                end = datetime.datetime.now()
                duration = (end-start).total_seconds()
                if self.verbose != 0: print("Train Time: {:5f}\tTest Accuracy: {:5f}\t\tETA: {:5f}s\t{}/{}".format(duration,acc,duration/count*(len(self.combinations)-count),count,len(self.combinations)))
        print("Search Complete\nBest params: {}\tHighest accuracy: {:5f}".format(str(self.best_params),highest))

In [58]:
model = GBDT(verbose=2,args={'min_samples_leaf': 2})
model.fit(onehot_ingredient_id_train, calorie_level_train, cv=1, analyze=False)

      Iter       Train Loss   Remaining Time 
         1           1.0790           42.30m
         2           1.0759           43.66m
         3           1.0732           43.00m
         4           1.0708           43.05m
         5           1.0687           43.23m
         6           1.0667           43.02m
         7           1.0648           42.50m
         8           1.0630           42.16m
         9           1.0614           41.62m
        10           1.0600           41.22m
        20           1.0482           37.05m
        30           1.0398           32.23m
        40           1.0331           27.17m
        50           1.0275           22.72m
        60           1.0228           18.19m
        70           1.0186           13.71m
        80           1.0148            9.17m
        90           1.0114            4.60m
       100           1.0083            0.00s
Train Accuracy: 0.506100


In [60]:
GBC_predictions = model.predict(onehot_ingredient_id_test)

In [62]:
print(precision_score(calorie_level_test, GBC_predictions,average='macro'))

0.5218689221959796


In [63]:
model.evaluate(onehot_ingredient_id_test, calorie_level_test)

Accuracy: 0.496536


0.49653605587187616